In [115]:
import clr
clr.AddReference("System.Runtime.InteropServices")
from System.Runtime.InteropServices import Marshal

#set the following path to the installed SAP2000 program directory
clr.AddReference(R'C:\Program Files\Computers and Structures\SAP2000 26\SAP2000v1.dll')
import SAP2000v1 as sap

#create API helper object
#helper = etabs.cHelper(etabs.Helper())
helper = sap.cHelper(sap.Helper())

try:
    #myETABSObject = etabs.cOAPI(helper.GetObject("CSI.ETABS.API.ETABSObject"))
    mySapObject = sap.cOAPI(helper.GetObject("CSI.SAP2000.API.SapObject"))
except:
    error_msg = "No running instance of the program found or failed to attach."
    print(error_msg)


#create SapModel object
SapModel = sap.cSapModel(mySapObject.SapModel)

N_mm_C = sap.eUnits.N_mm_C
kN_m_C = sap.eUnits.kN_m_C

display(mySapObject)
display(SapModel)

ret = SapModel.SetPresentUnits(N_mm_C)
ret

0

In [116]:
def get_selected_area(SapModel):
    GetSelected = {
        'NumberItems' : 0, 
        'ObjectType' : [], 
        'ObjectName' : [],
    }

    ret, NumberItems, ObjectType, ObjectName = SapModel.SelectObj.GetSelected(**GetSelected)

    ObjectType = list(ObjectType)
    ObjectName = list(ObjectName)
    selected_area = [obj_name for obj_type, obj_name in zip(ObjectType, ObjectName) if obj_type == 5]
    return selected_area
get_selected_area(SapModel)

def get_area_points(SapModel, area_name):
    GetPoints = {
        'Name' : area_name,
        'NumberPoints' : 0,
        'Point' : [],
    }
    ret, NumberPoints, Point = SapModel.AreaObj.GetPoints(**GetPoints)
    points = list(Point)
    return (area_name, points)


def set_area_local_axis(SapModel, area_point_tuple):
    area_name, points = area_point_tuple
    SetLocalAxesAdvanced = {
        'Name': area_name,
        'Active': True,
        'Plane2' : 31,
        'PlVectOpt' : 2,
        'PlCSys' : 'Global',
        'PlDir' : [],
        'PlPt' : [points[0], points[1]],
        'PlVect' : []
    }
    ret = SapModel.AreaObj.SetLocalAxesAdvanced(**SetLocalAxesAdvanced)
    return ret[0]

area_points = [get_area_points(SapModel, area) for area in get_selected_area(SapModel)]

_ = [set_area_local_axis(SapModel,area_point_tuple) for area_point_tuple in area_points] 


In [117]:
#refresh view
ret = SapModel.View.RefreshView(0, False)

SapModel = None

mySapObject = None
